In [19]:
import pyspark

In [20]:
#Create pyspark session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("filter operations").getOrCreate()
spark

In [21]:
#Read data
spark_df=spark.read.csv('test_file.csv',header=True,enforceSchema=True)
spark_df.show()

+----+----------+----+----------+------+
|Name|Department| Age|experience|salary|
+----+----------+----+----------+------+
|  AA|        D1|  25|         1|    70|
|  BB|        D1|  30|         3|   100|
|  CC|        D2|  35|         4|   110|
|  DD|        D3|  22|         1|    60|
|  EE|        D2|NULL|         3|    75|
|NULL|      NULL|NULL|      NULL|  NULL|
|  FF|      NULL|  44|         3|    73|
|  ZZ|        D3|  23|         2|    46|
|  KK|        D4|NULL|         2|    80|
+----+----------+----+----------+------+



In [22]:
#Cast Columns to the Correct Data Types:

from pyspark.sql.types import IntegerType, DoubleType

cols=["Age","experience","salary"]

for col in cols:
  spark_df = spark_df.withColumn(col, spark_df[col].cast(IntegerType()))
spark_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [23]:
spark_df.show()

+----+----------+----+----------+------+
|Name|Department| Age|experience|salary|
+----+----------+----+----------+------+
|  AA|        D1|  25|         1|    70|
|  BB|        D1|  30|         3|   100|
|  CC|        D2|  35|         4|   110|
|  DD|        D3|  22|         1|    60|
|  EE|        D2|NULL|         3|    75|
|NULL|      NULL|NULL|      NULL|  NULL|
|  FF|      NULL|  44|         3|    73|
|  ZZ|        D3|  23|         2|    46|
|  KK|        D4|NULL|         2|    80|
+----+----------+----+----------+------+



In [24]:
spark_df.fillna({'Age':30,'Department':'Unknown'}).show()

spark_df = spark_df.fillna({'Age':30,'Department':'Unknown'})

+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  AA|        D1| 25|         1|    70|
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  DD|        D3| 22|         1|    60|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|      NULL|  NULL|
|  FF|   Unknown| 44|         3|    73|
|  ZZ|        D3| 23|         2|    46|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



In [25]:
#fillimg with stats using imputer
from pyspark.ml.feature import Imputer

imputer=Imputer(inputCols=['Age'],outputCols=['Age']).setStrategy('mean')     #or mode, median
df_imputed=imputer.fit(spark_df).transform(spark_df)
df_imputed.show()

+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  AA|        D1| 25|         1|    70|
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  DD|        D3| 22|         1|    60|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|      NULL|  NULL|
|  FF|   Unknown| 44|         3|    73|
|  ZZ|        D3| 23|         2|    46|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



In [26]:
imputer=Imputer(inputCols=['salary'],outputCols=['salary']).setStrategy('mean')     #or mode, median
df_imputed=imputer.fit(df_imputed).transform(df_imputed)
df_imputed.show()

+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  AA|        D1| 25|         1|    70|
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  DD|        D3| 22|         1|    60|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|      NULL|    76|
|  FF|   Unknown| 44|         3|    73|
|  ZZ|        D3| 23|         2|    46|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



In [27]:
imputer=Imputer(inputCols=['experience'],outputCols=['experience']).setStrategy('mode')     #or mode, median
df_imputed=imputer.fit(df_imputed).transform(df_imputed)
df_imputed.show()

+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  AA|        D1| 25|         1|    70|
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  DD|        D3| 22|         1|    60|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|         3|    76|
|  FF|   Unknown| 44|         3|    73|
|  ZZ|        D3| 23|         2|    46|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



In [28]:
mode_value = df_imputed.groupBy("Department").count().orderBy("count", ascending=False).first()[0]
df_imputed = df_imputed.fillna({'Department': mode_value})
df_imputed.show()

+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  AA|        D1| 25|         1|    70|
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  DD|        D3| 22|         1|    60|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|         3|    76|
|  FF|   Unknown| 44|         3|    73|
|  ZZ|        D3| 23|         2|    46|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



**Filtering based on operators**

In [29]:
#filter based on operations ==,<,.. |, &, ~
df_imputed[df_imputed['Age']>=30].show()
# df_imputed.filter(df_imputed['Age']>=30).show()
# df_imputed.filter(df_imputed.Age>=30).show()


+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|         3|    76|
|  FF|   Unknown| 44|         3|    73|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



In [30]:
df_imputed[(df_imputed['Age']>=30) & (df_imputed['experience']==6)].show()


+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
+----+----------+---+----------+------+



**Groupby and aggregate**

In [31]:
df_imputed.show()

+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  AA|        D1| 25|         1|    70|
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  DD|        D3| 22|         1|    60|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|         3|    76|
|  FF|   Unknown| 44|         3|    73|
|  ZZ|        D3| 23|         2|    46|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



In [32]:
df_imputed.show()

+----+----------+---+----------+------+
|Name|Department|Age|experience|salary|
+----+----------+---+----------+------+
|  AA|        D1| 25|         1|    70|
|  BB|        D1| 30|         3|   100|
|  CC|        D2| 35|         4|   110|
|  DD|        D3| 22|         1|    60|
|  EE|        D2| 30|         3|    75|
|NULL|   Unknown| 30|         3|    76|
|  FF|   Unknown| 44|         3|    73|
|  ZZ|        D3| 23|         2|    46|
|  KK|        D4| 30|         2|    80|
+----+----------+---+----------+------+



In [33]:
#groupby
df_imputed.groupBy('Department').mean('Salary').show()   # alternatives sum,max,min,count

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|        D1|       85.0|
|        D3|       53.0|
|   Unknown|       74.5|
|        D2|       92.5|
|        D4|       80.0|
+----------+-----------+



In [34]:
df_imputed.groupBy('Department').min('Age').show()   # alternatives sum,max,min,count

+----------+--------+
|Department|min(Age)|
+----------+--------+
|        D1|      25|
|        D3|      22|
|   Unknown|      30|
|        D2|      30|
|        D4|      30|
+----------+--------+



In [35]:
#Aggregate
df_imputed.groupBy('Department').agg({'Age':'mean',
                                      'salary':'sum',
                                      'experience':'max'}).show()

+----------+-----------+---------------+--------+
|Department|sum(salary)|max(experience)|avg(Age)|
+----------+-----------+---------------+--------+
|        D1|        170|              3|    27.5|
|        D3|        106|              2|    22.5|
|   Unknown|        149|              3|    37.0|
|        D2|        185|              4|    32.5|
|        D4|         80|              2|    30.0|
+----------+-----------+---------------+--------+



In [36]:
# returns sum of salary for all

df_imputed.agg({'salary':'sum'}).show()

+-----------+
|sum(salary)|
+-----------+
|        690|
+-----------+

